### [!IMPORTANT]
**Data Migration Notice**: Arc's Virtual Cell Atlas data has migrated to the [Google Cloud Marketplace](https://console.cloud.google.com/marketplace/product/bigquery-public-data/arc-institute?project=gcp-public-data-arc-institute). 

**Note**: The new bucket is subject to [Requester Pays](https://docs.cloud.google.com/storage/docs/requester-pays). Users can access up to 2TB of data per month for free before fees apply.

Access to the current GCS buckets (`gs://arc-ctc-tahoe100/` and `gs://arc-scbasecount/`) will be deprecated on **March 31, 2026**. Please update your workflows to use the Google Marketplace bucket `gs://arc-institute-virtual-cell-atlas`.

# Summary

* This is a tutorial on using R (& Python via [Reticulate](https://rstudio.github.io/reticulate/)) 
  for accessing the scBaseCount dataset hosted by the Arc Institute.
* The data can be streamed or downloaded locally.
  * For small jobs (e.g., summarizing the some metadata), streaming is recommended.
  * For large jobs (e.g., training a model), downloading is recommended.
* See the [README](README.md#metadata) for a description of the obs metadata.


# Setup

### Installation

If needed, install the necessary dependencies.

You can use the [conda environment](../conda_envs/R.yml) provided in this git repository.

# Load libraries

In [1]:
# Load required libraries
library(dplyr)
library(arrow)
library(reticulate)
library(googleCloudStorageR)
os = import("os")
pd = import("pandas")
ad = import("anndata")
gcsfs = import("gcsfs")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘arrow’


The following object is masked from ‘package:utils’:

    timestamp




In [2]:
# set max print rows
options(repr.matrix.max.rows=4)

# Data location

In [3]:
# GCS bucket path
gcs_base_path = "gs://arc-institute-virtual-cell-atlas/scbasecount/2026-01-12"

In [4]:
# which STARsolo feature type to use?
feature_type = "GeneFull_Ex50pAS"

# List available files

Let's see what we have to work with!

In [5]:
# initialize GCS file system for reading data from GCS
fs = gcsfs$GCSFileSystem(
    token = Sys.getenv("GOOGLE_APPLICATION_CREDENTIALS")
)

In [6]:
# helper function
get_parquet_files = function(gcs_base_path, feature_type, target = NULL, endswith = NULL) {
  files = fs$glob(os$path$join(gcs_base_path, feature_type, "**"))
  

  if (!is.null(target)) {
    files = files[sapply(files, function(f) basename(f) == target)]
  } else if (!is.null(endswith)) {
    files = files[sapply(files, function(f) grepl(paste0(endswith, "$"), f))]
  }
  
  file_list = lapply(files, function(f) {
    parts = unlist(strsplit(f, "/"))
    c(parts[length(parts)-1], f)
  })
  
  file_df = as.data.frame(do.call(rbind, file_list), stringsAsFactors = FALSE)
  colnames(file_df) = c("organism", "file_path")
  
  return(file_df)
}

In [7]:
# function to read parquet files from GCS
read_data = function(file, n = NULL){
  if(!startsWith(file, "gs://")){
    file = paste0("gs://", file)
  }
  dataset = open_dataset(file, format = "parquet")
  if (is.null(n)) {
    dataset %>% collect() %>% as.data.frame()
  } else {
    dataset %>% head(n) %>% collect() %>% as.data.frame()
  }
}

## Parquet files

* Contain the obs metadata
* Similar to CSV files, but more efficient for large datasets

In [8]:
# set the path to the metadata files
gcs_path = file.path(gcs_base_path, "metadata")
gcs_path

[1] "gs://arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/metadata"

In [9]:
# list files
sample_pq_files = get_parquet_files(gcs_path, feature_type, "sample_metadata.parquet")
sample_pq_files 

organism,file_path
<chr>,<chr>
Arabidopsis_thaliana,arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/metadata/GeneFull_Ex50pAS/Arabidopsis_thaliana/sample_metadata.parquet
Bos_taurus,arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/metadata/GeneFull_Ex50pAS/Bos_taurus/sample_metadata.parquet
⋮,⋮
Taeniopygia_guttata,arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/metadata/GeneFull_Ex50pAS/Taeniopygia_guttata/sample_metadata.parquet
Zea_mays,arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/metadata/GeneFull_Ex50pAS/Zea_mays/sample_metadata.parquet


### Per-obs metadata

In [10]:
# list files
obs_pq_files = get_parquet_files(gcs_path, feature_type, "obs_metadata.parquet")
obs_pq_files 

organism,file_path
<chr>,<chr>
Arabidopsis_thaliana,arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/metadata/GeneFull_Ex50pAS/Arabidopsis_thaliana/obs_metadata.parquet
Bos_taurus,arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/metadata/GeneFull_Ex50pAS/Bos_taurus/obs_metadata.parquet
⋮,⋮
Taeniopygia_guttata,arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/metadata/GeneFull_Ex50pAS/Taeniopygia_guttata/obs_metadata.parquet
Zea_mays,arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/metadata/GeneFull_Ex50pAS/Zea_mays/obs_metadata.parquet


# h5ad files

In [11]:
gcs_path = file.path(gcs_base_path, "h5ad")
gcs_path

[1] "gs://arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/h5ad"

In [12]:
# list files
h5ad_files = get_parquet_files(gcs_path, feature_type, endswith=".h5ad")
h5ad_files 

organism,file_path
<chr>,<chr>
Arabidopsis_thaliana,arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/h5ad/GeneFull_Ex50pAS/Arabidopsis_thaliana/ERX6221408.h5ad
Arabidopsis_thaliana,arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/h5ad/GeneFull_Ex50pAS/Arabidopsis_thaliana/ERX6221414.h5ad
⋮,⋮
Zea_mays,arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/h5ad/GeneFull_Ex50pAS/Zea_mays/SRX8487984.h5ad
Zea_mays,arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/h5ad/GeneFull_Ex50pAS/Zea_mays/SRX8781690.h5ad


# Expore the observation metadata

* `obs` ≃ cell

### Per-sample

* Useful for quickly summarizing the per-sample metadata (a small file versus the entire obs metadata file; see below).
* For this tutorial, we will just read in a subset

In [13]:
# read the metadata files
num_organisms = 2
num_samples_per_organism = 10
sample_metadata = sample_pq_files %>%
  pull(file_path) %>%
  head(n = num_organisms) %>%
  lapply(read_data, n = num_samples_per_organism) %>%
  bind_rows()
sample_metadata

entrez_id,srx_accession,file_path,obs_count,lib_prep,tech_10x,cell_prep,organism,tissue,tissue_ontology_term_id,disease,disease_ontology_term_id,perturbation,cell_line,antibody_derived_tag,czi_collection_id,czi_collection_name
<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>
26779669,SRX19498703,gs://arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/h5ad/GeneFull_Ex50pAS/Arabidopsis_thaliana/SRX19498703.h5ad,2865,10x_Genomics,3_prime_gex,single_nucleus,Arabidopsis thaliana,whole flowers,UBERON:0000914,unsure,NA,unsure,not_applicable,no,NA,NA
20529885,SRX14437315,gs://arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/h5ad/GeneFull_Ex50pAS/Arabidopsis_thaliana/SRX14437315.h5ad,15115,10x_Genomics,3_prime_gex,single_cell,Arabidopsis thaliana,root tip,,unsure,NA,half MS (0.5x) growth medium,Col-0 cells,no,NA,NA
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
18826607,SRX13557914,gs://arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/h5ad/GeneFull_Ex50pAS/Bos_taurus/SRX13557914.h5ad,13304,10x_Genomics,3_prime_gex,single_cell,Bos taurus,cecum,UBERON:0001153,unsure,NA,unsure,unsure,no,NA,NA
18826600,SRX13557907,gs://arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/h5ad/GeneFull_Ex50pAS/Bos_taurus/SRX13557907.h5ad,12649,10x_Genomics,3_prime_gex,single_cell,Bos taurus,omasum,UBERON:0007362,unsure,NA,unsure,unsure,no,NA,NA


In [14]:
# samples per organism
sample_metadata %>% count(organism)

organism,n
<chr>,<int>
Arabidopsis thaliana,10
Bos taurus,10


In [15]:
# cell counts per sample
sample_metadata %>% pull(obs_count) %>% summary()

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    951    3856   11318    9642   13924   23594 

In [16]:
# cell count distribution per organism
sample_metadata %>%
  group_by(organism) %>%
  summarise(
    min = min(obs_count),
    max = max(obs_count),
    mean = mean(obs_count),
    median = median(obs_count),
    sd = sd(obs_count)
  )

organism,min,max,mean,median,sd
<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>
Arabidopsis thaliana,951,23594,7152.9,3595.5,7642.408
Bos taurus,4471,16351,12130.3,12609.0,3541.196


### Per-obs metadata

In [17]:
# read the metadata files
num_organisms = 2
num_obs_per_organism = 1000
obs_metadata = obs_pq_files %>%
  pull(file_path) %>%
  head(n = num_organisms) %>%
  lapply(read_data, n = num_obs_per_organism) %>%
  bind_rows()
obs_metadata

cell_barcode,SRX_accession,gene_count_Unique,umi_count_Unique,gene_count_UniqueAndMult-EM,umi_count_UniqueAndMult-EM,gene_count_UniqueAndMult-Uniform,umi_count_UniqueAndMult-Uniform,cell_type,cell_ontology_term_id
<chr>,<chr>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<chr>,<chr>
AAACCTGAGATATGCA,ERX6221408,814,1026,822,1031,822,1031,,
AAACCTGAGGCGATAC,ERX6221408,475,602,483,608,483,608,,
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
AGAGCAGAGCTCCACG,ERX13041271,3639,9688,3753,10335,3908,10335,,
AGAGCAGAGGATTTAG,ERX13041271,3333,8109,3447,8563,3568,8563,,


In [20]:
# gene count distribution
obs_metadata %>% pull(gene_count_Unique) %>% summary()

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    108     735    1834    2591    4410    8044 

In [21]:
# umi counts distribution
obs_metadata %>% pull(umi_count_Unique) %>% summary()

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  500.0   986.8  3111.0  7300.8 12792.5 44458.0 

#### Obs for particular samples

Let's get some obs metadata for particular samples

In [22]:
# re-print our selected samples
sample_metadata

entrez_id,srx_accession,file_path,obs_count,lib_prep,tech_10x,cell_prep,organism,tissue,tissue_ontology_term_id,disease,disease_ontology_term_id,perturbation,cell_line,antibody_derived_tag,czi_collection_id,czi_collection_name
<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>
26779669,SRX19498703,gs://arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/h5ad/GeneFull_Ex50pAS/Arabidopsis_thaliana/SRX19498703.h5ad,2865,10x_Genomics,3_prime_gex,single_nucleus,Arabidopsis thaliana,whole flowers,UBERON:0000914,unsure,NA,unsure,not_applicable,no,NA,NA
20529885,SRX14437315,gs://arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/h5ad/GeneFull_Ex50pAS/Arabidopsis_thaliana/SRX14437315.h5ad,15115,10x_Genomics,3_prime_gex,single_cell,Arabidopsis thaliana,root tip,,unsure,NA,half MS (0.5x) growth medium,Col-0 cells,no,NA,NA
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
18826607,SRX13557914,gs://arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/h5ad/GeneFull_Ex50pAS/Bos_taurus/SRX13557914.h5ad,13304,10x_Genomics,3_prime_gex,single_cell,Bos taurus,cecum,UBERON:0001153,unsure,NA,unsure,unsure,no,NA,NA
18826600,SRX13557907,gs://arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/h5ad/GeneFull_Ex50pAS/Bos_taurus/SRX13557907.h5ad,12649,10x_Genomics,3_prime_gex,single_cell,Bos taurus,omasum,UBERON:0007362,unsure,NA,unsure,unsure,no,NA,NA


In [23]:
# read the metadata files
target_organisms = sample_metadata %>% pull(organism) %>% unique() %>% gsub(" ", "_", .)
obs_metadata = obs_pq_files %>%
  filter(organism %in% target_organisms) %>%
  pull(file_path) %>%
  lapply(read_data) %>%
  bind_rows()

# join on SRX accession
obs_metadata = sample_metadata %>%
  inner_join(obs_metadata, by = c("srx_accession" = "SRX_accession"))
obs_metadata

entrez_id,srx_accession,file_path,obs_count,lib_prep,tech_10x,cell_prep,organism,tissue,tissue_ontology_term_id,⋯,czi_collection_name,cell_barcode,gene_count_Unique,umi_count_Unique,gene_count_UniqueAndMult-EM,umi_count_UniqueAndMult-EM,gene_count_UniqueAndMult-Uniform,umi_count_UniqueAndMult-Uniform,cell_type,cell_ontology_term_id
<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<lgl>,<chr>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<chr>,<chr>
26779669,SRX19498703,gs://arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/h5ad/GeneFull_Ex50pAS/Arabidopsis_thaliana/SRX19498703.h5ad,2865,10x_Genomics,3_prime_gex,single_nucleus,Arabidopsis thaliana,whole flowers,UBERON:0000914,⋯,NA,AAACCCACAAGACGGT,440,532,452,538,452,538,,
26779669,SRX19498703,gs://arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/h5ad/GeneFull_Ex50pAS/Arabidopsis_thaliana/SRX19498703.h5ad,2865,10x_Genomics,3_prime_gex,single_nucleus,Arabidopsis thaliana,whole flowers,UBERON:0000914,⋯,NA,AAACCCACAGAGATTA,407,545,417,553,419,553,,
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
18826600,SRX13557907,gs://arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/h5ad/GeneFull_Ex50pAS/Bos_taurus/SRX13557907.h5ad,12649,10x_Genomics,3_prime_gex,single_cell,Bos taurus,omasum,UBERON:0007362,⋯,NA,TTTGTTGTCGTTGCCT,438,730,470,798,501,798,,
18826600,SRX13557907,gs://arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/h5ad/GeneFull_Ex50pAS/Bos_taurus/SRX13557907.h5ad,12649,10x_Genomics,3_prime_gex,single_cell,Bos taurus,omasum,UBERON:0007362,⋯,NA,TTTGTTGTCTCTATAC,4831,17774,4920,18924,5092,18924,,


In [25]:
# summarize gene counts per organism
obs_metadata %>%
  group_by(organism) %>%
  summarise(
    min = min(gene_count_Unique),
    max = max(gene_count_Unique),
    mean = mean(gene_count_Unique),
    median = median(gene_count_Unique),
    sd = sd(gene_count_Unique)
  )

organism,min,max,mean,median,sd
<chr>,<int>,<int>,<dbl>,<int>,<dbl>
Arabidopsis thaliana,85,11860,1518.399,990,1394.469
Bos taurus,21,10506,2340.934,2228,1529.305


In [26]:
# summarize umi counts per organism
obs_metadata %>%
  group_by(organism) %>%
  summarise(
    min = min(umi_count_Unique),
    max = max(umi_count_Unique),
    mean = mean(umi_count_Unique),
    median = median(umi_count_Unique),
    sd = sd(umi_count_Unique)
  )

organism,min,max,mean,median,sd
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Arabidopsis thaliana,133,249412,4878.628,1783,9267.215
Bos taurus,500,291897,8674.563,5728,10245.028


# Load h5ad files

In [27]:
gcs_auth(json_file = Sys.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

In [34]:
gcs_base_path

[1] "gs://arc-institute-virtual-cell-atlas/scbasecount/2026-01-12"

In [43]:
# example
infile = file.path(gcs_base_path, "h5ad/GeneFull_Ex50pAS/Arabidopsis_thaliana/SRX16110574.h5ad")

# split bucket and path
bucket = unlist(strsplit(infile, "/"))[3]
path = paste(unlist(strsplit(infile, "/"))[-(1:3)], collapse = "/")

In [44]:
# create a temp file
temp_file = tempfile(fileext = ".h5ad")

In [45]:
# Download the file to a temporary location
gcs_get_object(object_name = path, bucket = bucket, saveToDisk = temp_file, overwrite = TRUE)

ℹ Downloading scbasecount/2026-01-12/h5ad/GeneFull_Ex50pAS/Arabidopsis_thaliana…

✔ Saved scbasecount/2026-01-12/h5ad/GeneFull_Ex50pAS/Arabidopsis_thaliana/SRX16…





[1] TRUE

In [46]:
# read in anndata object
adata = ad$read_h5ad(temp_file)
adata

AnnData object with n_obs × n_vars = 5906 × 31109
    obs: 'gene_count_Unique', 'umi_count_Unique', 'gene_count_UniqueAndMult-EM', 'umi_count_UniqueAndMult-EM', 'gene_count_UniqueAndMult-Uniform', 'umi_count_UniqueAndMult-Uniform', 'SRX_accession', 'cell_type', 'cell_ontology_term_id'
    var: 'gene_symbols'
    layers: 'UniqueAndMult-EM', 'UniqueAndMult-Uniform'

# Downloading files

You can use [gsutil](https://cloud.google.com/storage/docs/gsutil) to download any of the files in the bucket
and work with them locally.

For example:

```bash
gsutil cp gs://arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/h5ad/GeneFull_Ex50pAS/Homo_sapiens/ERX4319106.h5ad .
```

For large data transfers, it is better to use `gsutil rsync`:

```bash
gsutil -m rsync gs://arc-institute-virtual-cell-atlas/scbasecount/2026-01-12/h5ad/GeneFull_Ex50pAS/Callithrix_jacchus/ .
```

***

# sessionInfo

In [29]:
sessionInfo()

R version 4.2.3 (2023-03-15)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 22.04.5 LTS

Matrix products: default
BLAS/LAPACK: /home/nickyoungblut/miniforge3/envs/scbasecount-r/lib/libopenblasp-r0.3.29.so

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] googleCloudStorageR_0.7.0 reticulate_1.38.0        
[3] arrow_16.1.0              dplyr_1.1.4              

loaded via a namespace (and not attached):
 [1] zip_2.3.1         Rcpp_1.0.12       googleAuthR_2.0.2 pillar_1.9.0     
 [5] compiler_4.2.3    base64enc_0.1-3   tools_4.2.3       digest_0.6.36    
 [9] uuid_1.2-0        bit_4.0.5      